Part 1: A supervised learning approach to link-prediction (50%)
In the first part of this assignment you will develop a link-prediction algorithm based on the supervised learning methods you learned in this course. Link prediction can be framed as a classification problem, where an algorithm can be trained to predict the class of any possible link — existent (positive) or non-existent (negative). To tackle this problem you should follow some steps:

Feature engineering: First you should be able to import network data and define the features you believe are relevant to predict links in social networks. The data we will provide you consist of a social network where some links were removed. Each node will also be characterised by a categorical feature. Examples of important features might be the class that nodes belong to, or the number of common connections between nodes.
Defining a training, validation and test set: Once you have defined your feature space, you have to create your training, validation and test set. Your training set should be composed of positive examples (i.e., edges that exist in the network that was provided to you) and corresponding features. Your training set should also be composed of negative examples (i.e., edges that do not exist in the network provided). You will need to decide the ratio of positive and negative examples to include in your training set. You also have to decide about the split between training, validation and test set to train and select your best model. 
Model selection and validation: Once you have the required training and validation set, you have to select a model to be trained. You must use one of the methods discussed in the lectures (more complex models will not necessarily lead to higher performance, given the characteristics of the data we will provide you).
Application and test: Once you trained and validated your model, your solution should be able to decide if a (missing) link exists or not in a network. You will be welcome to submit your guesses to a Kaggle competition (details below). 
Note that we provide an auxiliary notebook (and auxiliary files) that can greatly help you in the previous steps. 


 

Data
The base dataset to use consists of a (synthetic) social network with 1500 nodes and 7333 links. This network was generated by removing ~10% of the links of an original network (only 6600 links are visible). Your task is to accurately guess whether a link given as an input belongs (or not) to the original network. All needed data can be found in the following file.

Network data: Each node is numbered from 0 to N-1, where N is the number of nodes in the network. The network structure is provided as a list of edges (edges_train.edgelist). In this file, each line corresponds to an edge (or link): int1,int2 where int1 and int2 correspond to the identification of each node connected by the edge. The network is assumed to be undirected (i.e., if a connection between nodes 1 and 2 exists in the file, the connection 2 to 1 is also assumed to exist).
Node data: Each node is characterised by a categorical feature that can take 5 possible values ['x', 'd', 'y', 'f', 'm', 'l']. These might constitute a sensitive attribute (e.g., political affiliation, personal preferences, demographic data...). The feature of each node is provided in a .csv file (attributes.csv). In this file, each line corresponds to a node and the value in a given line corresponds to the feature of the corresponding node. 
Final test data: The ultimate goal of your algorithm is to determine if a link exists or not in the original network. In file solutionInput.csv you are provided a list of links. You should be able to output, for each link, a prediction: 1 if you believe the links exists in the original network; 0 if you believe the links does not exist in the original network. solutionInput.csv contains a total of 1466 links: 733 exist in the original network (positive examples); 733 do not exist in the original network (negative examples). Your ultimate goal is to develop an algorithm that achieves high accuracy in this test data. In Lab5 you will already produce an example of output that your (assignment 2) program should provide (example of Lab 5 output here Download here).

In [ ]:
# import relevant libraries
import itertools
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score
from networkx.algorithms.community import greedy_modularity_communities

In [ ]:
# read in associated files
attri = pd.read_csv('attributes.csv', sep=',', index_col="ID")
sol_inp = pd.read_csv('solutionInput.csv', sep=',', index_col="ID")

In [ ]:
#view attributes.csv
attri.head()

In [ ]:
#view solutionInput.csv
sol_inp.head()

In [ ]:
# import edgelist
edge_list = nx.read_edgelist("edges_train.edgelist", data=False, nodetype = int, delimiter=',') # import

In [ ]:
#### COMMUNITY DETECTION ####

#the function will return a partition of nodes
c = list(greedy_modularity_communities(edge_list, resolution=0.8))

# Create a color map for each node based on its community
color_map = []

# Assign each community a unique color
colors = itertools.cycle(['r', 'g', 'b', 'c', 'm', 'y', 'orange', 'purple'])

# Create a dictionary to store which node belongs to which community
c_color = {}
for community, color in zip(c, colors):
    for node in community:
        c_color[node] = color

# Apply the color to each node based on its community
for node in edge_list.nodes:
    color_map.append(c_color[node])

# draw node network
nx.draw(edge_list, node_color=color_map)

In [ ]:
#### LINK PREDICTION ###

# Generate features from edge endpoints
# Input: getFeature(graph, node_i, node_j)
def getFeature(G, i, j):
    """
    Function to extract features for link prediction between two nodes (i, j).
    
    Parameters:
    G (Graph): NetworkX graph object
    i, j (nodes): Nodes for which features are calculated
    
    Returns:
    features (dict): A dictionary containing the calculated features
    """
    
    # Preferential Attachment
    pref_attach = list(nx.preferential_attachment(G, [(i, j)]))[0][2]
    
    # Common Neighbors
    common_neighbors = len(list(nx.common_neighbors(G, i, j)))
    
    # Jaccard Coefficient
    jaccard_coeff = list(nx.jaccard_coefficient(G, [(i, j)]))[0][2]
    
    # Adamic-Adar Index
    adamic_adar = list(nx.adamic_adar_index(G, [(i, j)]))[0][2]
    
    # Resource Allocation Index
    resource_alloc = list(nx.resource_allocation_index(G, [(i, j)]))[0][2]
    
    # Shortest Path (Handle exception if no path exists)
    try:
        shortest_path = nx.shortest_path_length(G, source=i, target=j)
    except nx.NetworkXNoPath:
        shortest_path = float('inf')  # Infinite if no path exists
    
    # Return a dictionary with the features
    features = {
        'preferential_attachment': pref_attach,
        'common_neighbors': common_neighbors,
        'jaccard_coefficient': jaccard_coeff,
        'adamic_adar': adamic_adar,
        'resource_allocation': resource_alloc,
        'shortest_path': shortest_path
    }
    
    return features

In [ ]:
# Iterating through rows
i = []
j = []
for index, row in sol_inp.iterrows():
    i = row['int1']
    j = row['int2']

In [ ]:
# call getFeature function on our edgelist
features = getFeature(edge_list,i,j)
print(features)